## Objective:

To predict which passengers survived the Titanic disaster using their features such as **age**, **sex**, **class**, etc.

## Step 1: Load the Data

Why: Before any analysis or modeling, we must understand the raw data format and content.

In [1]:
import pandas as pd

# Load the Titanic dataset
df = pd.read_csv("Dataset/titanic.csv")

# Show the first few rows
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


This helps us get a **feel for the data**, such as the features available, and begin identifying missing or unusable values.

## Step 2: Understand and Explore the Data

**Why:** EDA (Exploratory Data Analysis) helps uncover hidden patterns and feature relevance.

In [2]:
# Summary of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
# Check for null values
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

* We learn about missing values in **Age**, **Cabin**, and **Embarked**.
* Features like Name, Ticket, and Cabin may be less helpful in basic models.

In [4]:
# Statistical overview
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Step 3: Data Cleaning and Preprocessing

Data often contains missing values and irrelevant features that can negatively affect model performance. We clean it to ensure accuracy.

In [5]:
# First, make a working copy of the data
data = df.copy()

# Drop 'Cabin' due to too many missing values
data.drop(columns='Cabin', inplace=True)

# Fill missing 'Age' with the median (robust against outliers)
data['Age'] = data['Age'].fillna(data['Age'].median())

# Fill missing 'Embarked' with the most common value (mode)
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

## Step 4: Feature Engineering

**Why?**

We convert categorical values to numbers and add useful features to increase model accuracy.

In [6]:
# Convert 'Sex' and 'Embarked' to numerical values
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Drop non-numeric and ID-like features
data.drop(columns=['Name', 'Ticket', 'PassengerId'], inplace=True)

## Step 5: Model Training

**Why?**
  
We split the data into training and testing sets and train a machine learning model to learn survival patterns.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Separate features and target
X = data.drop('Survived', axis=1)
y = data['Survived']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

## Step 6: Results & Conclusion

In [8]:
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

Accuracy: 0.8268156424581006

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.88      0.86       105
           1       0.81      0.76      0.78        74

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179



## Conclusion:

* This basic pipeline yields solid accuracy (~83%).
* Key factors that affect survival: Sex, Age, and Passenger Class.
* You can improve this further with:
    * Hyperparameter tuning
    * Feature extraction (e.g. extracting titles from names)
    * Advanced models (e.g. XGBoost, neural networks)